In [1]:
library(bigrquery)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# Store the project ID
projectid = "yhcr-prd-phm-bia-core"

In [3]:
# load diabetes and cvd tables
# Diabetes
sql1 <- "SELECT *  FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.Diabetes`"
tb1 <- bq_project_query(projectid, sql1)
diabetes <-bq_table_download(tb1)

# CVD
sql2 <- "SELECT *  FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.All_CVD_First_Occurrence`"
tb2 <- bq_project_query(projectid, sql2)
cvd <-bq_table_download(tb2)

# Patient table
sql3 <- "SELECT * FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_AH.Patient_Denom`"
tb3 <- bq_project_query(projectid, sql3)
patient <- bq_table_download(tb3)

In [ ]:
covars <- read_csv("/home/jupyter/T2D_CVD_Study/Patient_Covariates_Table.csv")
covars <- covars %>% select(-'...1', -birth_datetime, -gender, -imd_decile, -ethnicity)

New names:
• `` -> `...1`
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 784186 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (20): ...1, person_id, gender, ethnicity, imd_decile, cholesterol, bmi,...
lgl   (4): date_cholesterol, closest_cholesterol, date_egfr, closest_egfr
date  (7): birth_datetime, death_datetime, date_bmi, date_smoking, date_bp, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
# join cvd and diabetes
dc <- left_join(diabetes, cvd)
dc <- dc %>%
    mutate_at(c('t1d_status', 't2d_status', 'any_diabetes', 'has_angina','has_cardiomyopathy', 'has_heart_failure', 'has_hypertension', 'has_ihd','has_mi', 'has_pvd', 'has_stroke', 'has_valve_disease'), ~replace_na(.,0))
head(dc)
sum(duplicated(dc))
sum(duplicated(dc$person_id))

Joining with `by = join_by(person_id)`


person_id,age_t1d_diag,date_t1d_diag,t1d_status,age_t2d_diag,date_t2d_diag,t2d_status,age_earliest_diab,earliest_diabetes_diag,any_diabetes,⋯,date_ihd_diagnosis,has_ihd,date_mi_diagnosis,has_mi,date_pvd_diagnosis,has_pvd,date_stroke_diagnosis,has_stroke,date_valve_disease_diagnosis,has_valve_disease
<int>,<int>,<date>,<int>,<int>,<date>,<int>,<int>,<date>,<int>,⋯,<date>,<int>,<date>,<int>,<date>,<int>,<date>,<int>,<date>,<int>
13548546,60,2007-01-11,1,NA,NA,0,60,2007-01-11,1,⋯,NA,0,NA,0,NA,0,NA,0,NA,0
12975170,80,2020-04-28,1,NA,NA,0,80,2020-04-28,1,⋯,NA,0,NA,0,NA,0,2013-03-05,1,NA,0
13450523,36,2011-10-13,1,NA,NA,0,36,2011-10-13,1,⋯,NA,0,NA,0,NA,0,NA,0,NA,0
12431500,75,2011-08-16,1,NA,NA,0,75,2011-08-16,1,⋯,2012-04-03,1,NA,0,NA,0,NA,0,NA,0
13235332,47,2007-07-05,1,NA,NA,0,47,2007-07-05,1,⋯,NA,0,NA,0,NA,0,NA,0,NA,0
12805324,39,2008-04-08,1,NA,NA,0,39,2008-04-08,1,⋯,NA,0,NA,0,NA,0,NA,0,NA,0


[1] 0

[1] 0

In [6]:
# Join death datetime onto dc
dc <- left_join(dc, patient, by = "person_id")
head(dc)
sum(duplicated(dc))
sum(duplicated(dc$person_id))

person_id,age_t1d_diag,date_t1d_diag,t1d_status,age_t2d_diag,date_t2d_diag,t2d_status,age_earliest_diab,earliest_diabetes_diag,any_diabetes,⋯,has_pvd,date_stroke_diagnosis,has_stroke,date_valve_disease_diagnosis,has_valve_disease,birth_datetime,death_datetime,gender,ethnicity,imd_decile
<int>,<int>,<date>,<int>,<int>,<date>,<int>,<int>,<date>,<int>,⋯,<int>,<date>,<int>,<date>,<int>,<date>,<date>,<int>,<int>,<int>
13548546,60,2007-01-11,1,NA,NA,0,60,2007-01-11,1,⋯,0,NA,0,NA,0,1946-11-15,2008-07-15,1,1,5
12975170,80,2020-04-28,1,NA,NA,0,80,2020-04-28,1,⋯,0,2013-03-05,1,NA,0,1939-11-15,2020-05-15,1,1,1
13450523,36,2011-10-13,1,NA,NA,0,36,2011-10-13,1,⋯,0,NA,0,NA,0,1975-08-15,2018-12-15,1,1,4
12431500,75,2011-08-16,1,NA,NA,0,75,2011-08-16,1,⋯,0,NA,0,NA,0,1936-05-15,2013-01-15,1,1,1
13235332,47,2007-07-05,1,NA,NA,0,47,2007-07-05,1,⋯,0,NA,0,NA,0,1960-03-15,2013-09-15,1,1,1
12805324,39,2008-04-08,1,NA,NA,0,39,2008-04-08,1,⋯,0,NA,0,NA,0,1968-08-15,2017-03-15,1,2,1


[1] 0

[1] 0

In [7]:
# Create variable for time between:

dc <- dc %>%
    mutate(date_earliest_cvd = pmin(date_angina_diagnosis, date_cardiomyopathy_diagnosis,
                               date_heart_failure_diagnosis, date_hypertension_diagnosis,
                              date_ihd_diagnosis, date_mi_diagnosis, date_pvd_diagnosis,
                              date_stroke_diagnosis, date_valve_disease_diagnosis, na.rm = TRUE),
# Diabetes and CVD
    diab_cvd_time = abs(difftime(earliest_diabetes_diag, date_earliest_cvd, units = "days")),
# CVD and Death
          cvd_death_time = abs(difftime(date_earliest_cvd, death_datetime, units = "days")),
          has_cvd = ifelse(has_angina == 1 | has_cardiomyopathy == 1 | has_heart_failure == 1 |
                           has_hypertension == 1 | has_ihd == 1 | has_mi == 1 | has_pvd == 1 |
                           has_stroke == 1 | has_valve_disease == 1, 1,0)) %>%
        mutate(diab_cvd_time = as.integer(diab_cvd_time),
               diab_death_time = as.integer(diab_death_time),
               cvd_death_time = as.integer(cvd_death_time))

head(dc)

person_id,age_t1d_diag,date_t1d_diag,t1d_status,age_t2d_diag,date_t2d_diag,t2d_status,age_earliest_diab,earliest_diabetes_diag,any_diabetes,⋯,has_valve_disease,birth_datetime,death_datetime,gender,ethnicity,imd_decile,date_earliest_cvd,diab_cvd_time,cvd_death_time,has_cvd
<int>,<int>,<date>,<int>,<int>,<date>,<int>,<int>,<date>,<int>,⋯,<int>,<date>,<date>,<int>,<int>,<int>,<date>,<int>,<int>,<dbl>
13548546,60,2007-01-11,1,NA,NA,0,60,2007-01-11,1,⋯,0,1946-11-15,2008-07-15,1,1,5,NA,NA,NA,0
12975170,80,2020-04-28,1,NA,NA,0,80,2020-04-28,1,⋯,0,1939-11-15,2020-05-15,1,1,1,2013-03-05,2611,2628,1
13450523,36,2011-10-13,1,NA,NA,0,36,2011-10-13,1,⋯,0,1975-08-15,2018-12-15,1,1,4,NA,NA,NA,0
12431500,75,2011-08-16,1,NA,NA,0,75,2011-08-16,1,⋯,0,1936-05-15,2013-01-15,1,1,1,2012-01-04,141,377,1
13235332,47,2007-07-05,1,NA,NA,0,47,2007-07-05,1,⋯,0,1960-03-15,2013-09-15,1,1,1,NA,NA,NA,0
12805324,39,2008-04-08,1,NA,NA,0,39,2008-04-08,1,⋯,0,1968-08-15,2017-03-15,1,2,1,NA,NA,NA,0


In [8]:
# Join covariates to diabetes dataset
dc <- left_join(dc, covars, by = c("person_id"))

In [14]:
# Import and Search Death certificate dataset for ICD10 CVD death codes
# Angina
sql8 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_2 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_3 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_4 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_5 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_6 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_7 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_8 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_9 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_10 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_11 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_12 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_13 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_14 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')
OR s_cod_code_15 IN ('I200','I248','I209','I20','A360','A691','B085','B27','D70X','I739','J020','J312','K122','I208','I201')"
tb8 <- bq_project_query(projectid, sql8)
angina_dcert <- bq_table_download(tb8)

# Cardiomyopathy
sql9 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_2 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_3 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_4 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_5 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_6 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_7 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_8 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_9 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_10 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_11 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_12 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_13 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_14 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')
OR s_cod_code_15 IN ('E854D','I431A','I255','I429','I42','I090','I11','I251','I43A','O903','O994','Q248','I423','I421','I428','I425','I422','I420','B572D','B570D','I981A','I412A','I428D','E90XA','E740D','E763D','I528A','I438A','G111D','G711D','I427','I515','I430A','I432A','G710','I518','G710D')"
tb9 <- bq_project_query(projectid, sql9)
cardiomyopathy_dcert <- bq_table_download(tb9)

# Heart Failure
sql11 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_2 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_3 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_4 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_5 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_6 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_7 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_8 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_9 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_10 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_11 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_12 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_13 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_14 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')
OR s_cod_code_15 IN ('I099','I509','I50','I110','I130','I132','I971','I978','O291','O742','O754','O891','P290','T818','E059D','I500','I10X','I501','I018','I020','I098','I970')"
tb11 <- bq_project_query(projectid, sql11)
heart_failure_dcert <- bq_table_download(tb11)

# Hypertension
sql19 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_2 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_3 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_4 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_5 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_6 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_7 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_8 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_9 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_10 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_11 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_12 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_13 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_14 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')
OR s_cod_code_15 IN ('I158','O16X','I21','I22','I23','I24','I25','I270','O10','O11X','O13X','O14','O15','P292','I12','I13','P000','I129','I159','I15','O104','I150','I152','I151','O12','O100','O102','O109','O121','O141','O159','O152','O150','O149','R688','O151','O142','O140')"
tb19 <- bq_project_query(projectid, sql19)
hypertension_dcert <- bq_table_download(tb19)

# IHD
sql97 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_2 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_3 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_4 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_5 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_6 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_7 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_8 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_9 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_10 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_11 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_12 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_13 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_14 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')
OR s_cod_code_15 IN ('I219','I214','I212','I249','I258','I250','I253','I254','I259','I233','I256','P294','I232')"
tb97 <- bq_project_query(projectid, sql97)
ihd_dcert <- bq_table_download(tb97)

# MI
sql91 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_2 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_3 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_4 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_5 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_6 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_7 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_8 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_9 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_10 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_11 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_12 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_13 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_14 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')
OR s_cod_code_15 IN ('I210','I220','I211','I221','I229','I228','I213','I252','A520D','I520A')"
tb91 <- bq_project_query(projectid, sql91)
mi_dcert <- bq_table_download(tb91)

# PVD
sql92 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_2 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_3 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_4 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_5 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_6 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_7 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_8 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_9 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_10 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_11 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_12 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_13 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_14 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')
OR s_cod_code_15 IN ('I738','E145D','E105D','E115D','E125D','E135D','O243D','O244D','O249D','I792A','I73')"
tb92 <- bq_project_query(projectid, sql92)
pvd_dcert <- bq_table_download(tb92)

# Stroke
sql93 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_2 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_3 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_4 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_5 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_6 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_7 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_8 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_9 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_10 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_11 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_12 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_13 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_14 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')
OR s_cod_code_15 IN ('I619','I61','I691','I618','G459','G45','G451','G450','R55X','I603','I611','I663D','G463A','I662D','G462A','I614','P526','S068','I631','I600','I639','Z866','I612','I64X','I610','I690','I692','I693','I694','I63','I669','I66','I633','I634','I635','G465A','I639D','I609','I60','P103','P525','I668','G452','I601','I660D','G460A','I613','I630','I632','G466A','G453','I616','G458','I606','I607','Z867','I615','G454','R413','I678','I636','I604','I661D','G461A','G467A','I638','I608','I605','P101','P524','G464A','I602','I679','I67','G46','I68','I671','I672','S062','S063','P10','P52','I658')"
tb93 <- bq_project_query(projectid, sql93)
stroke_dcert <- bq_table_download(tb93)

# Valve Disease
sql94 <- "SELECT * 
FROM `yhcr-prd-phm-bia-core.CB_FDM_DeathCertificates.tbl_NEC_Deaths_QWO_20230817_RELEASE`
WHERE s_cod_code_1 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_2 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_3 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_4 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_5 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_6 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_7 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_8 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_9 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_10 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_11 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_12 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_13 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_14 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')
OR s_cod_code_15 IN ('I059','I05','I080','I081','I083','I34','I050','I051','I052','I011','I342','I069','I06','I060','I062','I082','I061','I078','I070','I072','I071','I079','I088','I234','I235','I340','Q233','I349','I359','I35','I391A','I351','Q231','I350','I352','Q230','I369','I36','I361','I360','I362','I07','I379','I37','I393A','Q221','Q222','Q223','I371','I370','I372','I089','I08','I394A','I38X','I398A','I512','I058','I068','I348','I358','I368','I378','I390A','I392A','Q220','Q224','Q225','Q232','Q238','Q244','Q253','Q228','Q229','T828','I341','Q239','Q23','I710','Q234','Q226','T820','I712','I711','Q254','I790A','Q208','Q210','I02','I511','I33','A520A','I330')"
tb94 <- bq_project_query(projectid, sql94)
valve_disease_dcert <- bq_table_download(tb94)

In [29]:
# Keep relevant vars and derive has_*CVD*_death var
# Angina
angina <- angina_dcert %>%
    mutate(has_angina_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_angina_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# Cardiomyopathy
cardiomyopathy <- cardiomyopathy_dcert %>%
    mutate(has_cardiomyopathy_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_cardiomyopathy_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# Heart Failure
heart_failure <- heart_failure_dcert %>%
    mutate(has_heart_failure_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_heart_failure_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# Hypertension
hypertension <- hypertension_dcert %>%
    mutate(has_hypertension_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_hypertension_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# IHD
ihd <- ihd_dcert %>%
    mutate(has_ihd_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_ihd_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# MI
mi <- mi_dcert %>%
    mutate(has_mi_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_mi_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# PVD
pvd <- pvd_dcert %>%
    mutate(has_pvd_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_pvd_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# Stroke
stroke <- stroke_dcert %>%
    mutate(has_stroke_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_stroke_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

# Valve Disease
valve_disease <- valve_disease_dcert %>%
    mutate(has_valve_disease_dc = 1) %>%
    select(person_id, gender = sex, dec_agec, has_valve_disease_dc) %>%
    mutate(gender = as.integer(gender)) %>%
    mutate(person_id = as.double(person_id))

In [18]:
# Keep relevant vars in diabetes dataset
dm <- dc %>%
    select(person_id, gender, ethnicity, imd_decile, birth_datetime, death_datetime.x, death_datetime.y, age_t1d_diag, date_t1d_diag, t1d_status, age_t2d_diag, date_t2d_diag, t2d_status, age_earliest_diab, earliest_diabetes_diag, 
           any_diabetes, diab_death_time, cvd_death_time, diab_cvd_time, has_cvd)

In [45]:
# Join death certificate datasets to dm data
dm_dcert <- left_join(dm, angina, by = c("person_id", "gender")) 
dm_dcert <- left_join(dm_dcert, cardiomyopathy, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, heart_failure, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, hypertension, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, ihd, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, mi, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, pvd, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, stroke, by = c("person_id", "gender", "dec_agec"))
dm_dcert <- left_join(dm_dcert, valve_disease, by = c("person_id", "gender", "dec_agec"))

In [54]:
# Replace NAs across has_* vars
dm_dcert <- dm_dcert %>%
    mutate_at(c('has_angina_dc','has_cardiomyopathy_dc', 'has_heart_failure_dc', 'has_hypertension_dc', 'has_ihd_dc', 'has_mi_dc', 'has_pvd_dc', 'has_stroke_dc', 'has_valve_disease_dc'), ~replace_na(.,0)) %>%
    mutate(sum_cvd_dc = rowSums(across(c("has_angina_dc", "has_cardiomyopathy_dc", "has_heart_failure_dc", "has_hypertension_dc", "has_ihd_dc", "has_mi_dc", "has_pvd_dc", "has_stroke_dc", "has_valve_disease_dc")))) %>%
    mutate(has_cvd_dc = ifelse(sum_cvd_dc >= 1,1,0))

In [64]:
table(dm_dcert$sum_cvd_dc)
table(dm_dcert$has_cvd_dc)
table(dm_dcert$has_cvd_dc, dm_dcert$has_cvd)
nrow(dm_dcert)
table(dm_dcert$any_diabetes)
sum(duplicated(dm_dcert))
table(dm_dcert$any_diabetes, dm_dcert$has_cvd_dc)
table(dm_dcert$has_cvd)


    0     1     2     3     4 
43957   125   132    56     7 


    0     1 
43957   320 

   
        0     1
  0 28145 15812
  1    70   250

[1] 44277


    0     1 
   27 44250 

[1] 0

   
        0     1
  0    26     1
  1 43931   319


    0     1 
28215 16062 

In [1]:
# Deceased diabetic patients (n = 10528) who have a CVD on their death certificate 
(319/10528)*100

[1] 3.030015